# Introduction

- Ce notebook regroupe les étapes du préprocessing nécessaire pour passer du texte brut à un format de données exploitables par un RNN.
- Il doit être impérativement exécuté avant le notebook `generation_model.ipynb`
- Le corpus initial a volontairement été réduit pour ne garder que les discours prononcés par des candidats, et pour que l'on puisse vous envoyer un fichier peu volumineux :)

# Imports

In [ ]:
import json
import unidecode
import re
import string

import pandas as pd
import numpy as np

from re import IGNORECASE
from tensorflow.keras.utils import to_categorical

# Data

In [ ]:
speeches_df = pd.read_csv("../../datasets/speeches/train_speeches.csv")
speeches_df.head()

In [ ]:
print("Nombre de discours :", speeches_df.shape[0])
print("Nombre de caractères du corpus (tous discours confondus) :", sum(speeches_df["discours"].str.len()))

# Preprocessing

Afin de réduire la taille de notre vocabulaire (et donc le nombre de paramètres de notre modèle), on se propose de nettoyer le texte comme suit :
- Passer en minuscules et en caractères non accentués
- Remplacer toutes les ponctuations par des points
- Remplacer tous les caractères spéciaux par des espaces
- Remplacer les espaces multiples par un seul espace

In [ ]:
def clean_text(text):
    unaccented_text = unidecode.unidecode(text.lower())                                       # Minuscules, accents
    wihtout_special_chars = re.sub('["#$%&()*+-/:<=>@[\\]^_`{|}~\t\n]', ' ', unaccented_text) # Caractères spéciaux
    without_punct = re.sub('[!?;]', '.', wihtout_special_chars)                               # Ponctuation
    without_extra_spaces = re.sub(' +', ' ', without_punct)                                   # Espaces en trop
    return without_extra_spaces

print(" - Exemple avant nettoyage :")
print(speeches_df['discours'].iloc[0][:529])
print()
print(" - Exemple après nettoyage :")
print(clean_text(speeches_df['discours'].iloc[0][:529]))

In [ ]:
clean_df = pd.DataFrame(speeches_df['discours'].apply(clean_text))
print()
print("Après nettoyage de tous les discours :")
clean_df.head()

In [ ]:
def vocab_size(clean_text):
    return len(set(clean_text))

print("Taille du vocabulaire (nombre de caractères distincts dans tout le corpus) :", max(clean_df['discours'].apply(vocab_size).values))

- Nous avons maintenant un texte nettoyé, avec un vocabulaire restreint. Toutefois, les modèles de machine learning traitent des données numériques et non du texte. On doit donc trouver une représentation numérique pour chaque caractère. La façon la plus simple de faire cela est le one-hot encoding.

- Le one-hot encoding consiste à représenter chaque caractère comme un vecteur de taille 39 (taille de notre vocabulaire). Ce vecteur vaudra 1 à la position correspondant au caractère en question, et 0 partout ailleurs.

- Dans la cellule suivante, on crée notre vocabulaire constitué des caractères distincts qu'on a gardés après nettoyage. On crée également un mapping indice unique -> caractère et le mapping inverse caratère -> indice unique. Ces mappings serviront à créer les fonctions one_hot_encode et one_hot_decode.

In [ ]:
vocab = list(string.digits) + list(string.ascii_lowercase) + ['.', ' ', '\'']

index_to_char = {index: vocab[index] for index in range(len(vocab))}               # Mapping indice -> caractère
char_to_index = {char: index for index, char in index_to_char.items()}             # Mapping caractère -> indice

def one_hot_encode(text):
    indices = [char_to_index[c] for c in text]
    one_hot_encoding = np.zeros((len(indices), len(char_to_index)))
    for i in range(len(indices)):
        one_hot_encoding[i, indices[i]] = 1
    return one_hot_encoding

def one_hot_decode(encoding):
    result = ""
    for e in encoding:
        result = result + index_to_char[list(e).index(1)]
    return result

print("Vocabulaire :", vocab)
print()
print("Mapping indice -> caractère :", index_to_char)
print()
print("Mapping caractère -> indice :", char_to_index)

Testons nos fonctions d'encodage / décodage sur un exemple de discours :

In [ ]:
sample_speech = clean_df['discours'].iloc[0][:521]
encoded_speech = one_hot_encode(sample_speech)
decoded_code = one_hot_decode(encoded_speech)

print(" - Exemple de discours :")
print(sample_speech)
print("-> Longueur du discours :", len(sample_speech))
print("\n - Après one-hot encoding :")
print(encoded_speech)
print("-> Taille de la matrice d'encodage :", encoded_speech.shape)
print("\n - Décodage du one-hot encoding:")
print(decoded_code)
print("-> Longueur du discours décodé :", len(decoded_code))

In [ ]:
print("Encodage de tous les discours... \n")

encoded_texts = []
for text in clean_df["discours"]:
    encoded_texts.append(one_hot_encode(text))

print(encoded_texts)

Dans la bibliothèque Keras, un modèle de type RNN travaille sur des séquences de taille fixe. 

Il attend en entrée une matrice à 3 dimensions : (nombre d'échantillons, longueur des séquences, features)

Dans notre cas, chaque caractère est représenté par 39 features (résultat du one-hot encoding). La dernière dimension est donc satisfaite.

Toutefois, tous les discours n'ont pas la même longueur. La fonction suivante va donc permettre de découper une séquence de taille arbitraire en sous-séquences de taille fixe qui vont servir d'inputs au modèle. Par la même occasion, elle nous permettra de récupérer les targets associées à ces inputs, à savoir le caractère à prédire pour chaque input.

Le dernier argument de la fonction, skip, permet de contrôler l'overlap qu'on souhaite autoriser entre les sous-séquences.
- skip = 1 permet d'extraire toutes les sous-séquences possibles, ce qui fournit beaucoup de données mais augmente le risque d'overfitting.
- skip = s permet de "sauter" s caractères à chaque extraction de sous-séquences, ce qui réduit l'overlap mais fournit moins d'inputs au modèle.

In [ ]:
def cut_to_sequences(encoded_text, seq_length, skip):
    inputs, targets = [], []
    for i in range(0, len(encoded_text)-seq_length, skip):
        x = list(encoded_text[i:(i+seq_length)])
        y = encoded_text[i + seq_length]
        inputs.append(x)
        targets.append(y)
    return np.array(inputs), np.array(targets)

print("Taille de la matrice d'encodage du premier discours :", encoded_texts[0].shape)
cut_X, cut_y = cut_to_sequences(encoded_texts[0], 50, 5)
print()
print("Inputs après découpage en sous-séquences de taille 50 avec un skip de 3 :", cut_X.shape)
print("Targets associées :", cut_y.shape)

On applique le même traitement à tous les discours. 

Ensuite, on doit concaténer toutes les sous-séquences issues de ce découpage pour former une seule grande matrice de dimensions (nombre d'échantillons, longueur des séquences, features). Pour l'exercice, on ne gardera que les séquences correspondant aux 60 premiers discours car cette opération est consommatrice en mémoire et cette implémentation n'est pas la plus optimale...

In [ ]:
seq_data = [cut_to_sequences(encoded_text, 50, 5) for encoded_text in encoded_texts[:60]]

inputs = []
targets = []

for x, y in seq_data:
    inputs.append(x)
    targets.append(y)
    
inputs_array = np.concatenate(inputs)
targets_array = np.concatenate(targets)

In [ ]:
print("Taille de la matrice finale d'inputs :", inputs_array.shape)
print("Taille de la matrice finale de targets :", targets_array.shape)

On sauvegarde le résultat de notre preprocessing dans des fichiers en vue de l'utiliser plus tard pour le modèle.

In [ ]:
np.save("../../datasets/speeches/speeches_inputs.npy", inputs_array)
np.save("../../datasets/speeches/speeches_targets.npy", targets_array)